In [1]:
from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from numpy import argmax, array_equal
import matplotlib.pyplot as plt
from keras.models import Model
from random import randint
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
import random
from sklearn.model_selection import train_test_split
import keras.layers as layers


2023-11-07 12:56:54.409135: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 12:56:57.649434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
images = []
i=0
for fname in os.listdir('./images/images'):
    img = load_img(f'./images/images/{fname}')
    img=img.resize([100,100])
    images.append(img_to_array(img))


In [3]:
size = 100 * 100 * 3


In [4]:
images=np.array(images)
images=images/255.00
train_x,val_x=train_test_split(images,test_size=0.2)


In [5]:
train_x.shape


(14027, 100, 100, 3)

In [6]:
val_x.shape


(3507, 100, 100, 3)

In [7]:
def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """

    noise_factor = 0.4
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)


In [8]:
noisy_train_data = noise(train_x)
noisy_test_data = noise(val_x)


In [9]:
input = layers.Input(shape=(100, 100, 3))

# Encoder
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()


2023-11-07 12:59:11.689235: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 12:59:12.985032: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 12:59:12.985214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 12:59:13.015830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 12:59:13.016013: I tensorflow/compile

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 50, 50, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 25, 25, 32)        0     

In [10]:
autoencoder.fit(
    x=train_x,
    y=train_x,
    epochs=10,
    batch_size=32,
    shuffle=True,
    validation_data=(val_x, val_x),
)


2023-11-07 12:59:15.019205: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1683240000 exceeds 10% of free system memory.
2023-11-07 12:59:16.370443: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1683240000 exceeds 10% of free system memory.
2023-11-07 12:59:22.165990: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1683240000 exceeds 10% of free system memory.
2023-11-07 12:59:22.929974: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1683240000 exceeds 10% of free system memory.


Epoch 1/10


2023-11-07 12:59:31.812427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-07 12:59:33.479112: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-07 12:59:36.542258: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9ee4606a30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-07 12:59:36.542346: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2023-11-07 12:59:37.347094: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-07 12:59:38.601099: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-07 12:59:38.812951: I ./tensorflow/compiler/jit/device_

439/439 [==============================] - ETA: 0s - loss: 0.5335

2023-11-07 12:59:56.597672: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 420840000 exceeds 10% of free system memory.


439/439 [==============================] - 33s 44ms/step - loss: 0.5335 - val_loss: 0.5121
Epoch 2/10
439/439 [==============================] - 20s 46ms/step - loss: 0.5114 - val_loss: 0.5099
Epoch 3/10
439/439 [==============================] - 16s 37ms/step - loss: 0.5100 - val_loss: 0.5089
Epoch 4/10
439/439 [==============================] - 16s 37ms/step - loss: 0.5091 - val_loss: 0.5085
Epoch 5/10
439/439 [==============================] - 17s 38ms/step - loss: 0.5083 - val_loss: 0.5075
Epoch 6/10
439/439 [==============================] - 17s 38ms/step - loss: 0.5079 - val_loss: 0.5070
Epoch 7/10
439/439 [==============================] - 17s 39ms/step - loss: 0.5076 - val_loss: 0.5072
Epoch 8/10
439/439 [==============================] - 17s 38ms/step - loss: 0.5073 - val_loss: 0.5065
Epoch 9/10
439/439 [==============================] - 17s 38ms/step - loss: 0.5072 - val_loss: 0.5064
Epoch 10/10
439/439 [==============================] - 17s 38ms/step - loss: 0.5070 - val_los

In [14]:
predictions = autoencoder.predict(val_x)


2023-11-07 13:04:41.511232: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 401.34MiB (rounded to 420840192)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-07 13:04:41.511307: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-11-07 13:04:41.511325: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 39, Chunks in use: 39. 9.8KiB allocated for chunks. 9.8KiB in use in bin. 1.7KiB client-requested in use in bin.
2023-11-07 13:04:41.511332: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-11-07 13:04:41.511339: I tensorflow/tsl/framework/bfc_allocator

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

0600 of size 256 next 3
2023-11-07 13:04:41.511478: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800700 of size 256 next 5
2023-11-07 13:04:41.511482: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800800 of size 256 next 6
2023-11-07 13:04:41.511485: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800900 of size 256 next 4
2023-11-07 13:04:41.511489: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800a00 of size 256 next 7
2023-11-07 13:04:41.511493: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800b00 of size 256 next 12
2023-11-07 13:04:41.511496: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800c00 of size 256 next 15
2023-11-07 13:04:41.511500: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800d00 of size 256 next 17
2023-11-07 13:04:41.511504: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 703800e00 of size 6144 next 8
2023-11-07 13:04:41.511508: I tensorflow/tsl